In [ ]:
from controller import Display, Robot, Camera, Keyboard
from vehicle import Car, Driver
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
from skimage.feature import hog
import joblib

# --- Constantes de configuración ---
MODEL_PATH = 'modelo_conduccion.h5'  # Full path to model file
PEDESTRIAN_MODEL_PATH = 'controllers/proyectoFinalModel/pedestrian_detector.joblib'
IMG_WIDTH = 320
IMG_HEIGHT = 160

#Getting image from camera
def get_image(camera):
    raw_image = camera.getImage()
    image = np.frombuffer(raw_image, np.uint8).reshape(
        (camera.getHeight(), camera.getWidth(), 4)
    )
    return image

#Image processing
def greyscale_cv2(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_img

#Display image
def display_image(display, image):
    # Image to display
    image_rgb = np.dstack((image, image,image,))
    # Display image
    image_ref = display.imageNew(
        image_rgb.tobytes(),
        Display.RGB,
        width=image_rgb.shape[1],
        height=image_rgb.shape[0],
    )
    display.imagePaste(image_ref, 0, 0, False)

#initial angle and speed
manual_steering = 0
steering_angle = 0
angle = 0.0
initial_speed = 20 # Define initial_speed as a constant for the normal cruising speed
speed_increment = 0.5 # Speed increment when no obstacles detected
MAX_SPEED = 25 # Maximum allowed speed

# set target speed (this function is currently not used in main loop)
def set_speed(kmh):
    # This function is not directly used by the main loop's speed control logic,
    # as `target_speed` is managed directly within `main`.
    # Keeping it for potential future direct use, but `current_speed` isn't globally updated here anymore.
    pass

#update steering angle
def set_steering_angle(wheel_angle):
    global angle, steering_angle
    # Check limits of steering
    if (wheel_angle - steering_angle) > 0.1:
        wheel_angle = steering_angle + 0.1
    if (wheel_angle - steering_angle) < -0.1:
        wheel_angle = steering_angle - 0.1
    steering_angle = wheel_angle

    # limit range of the steering angle
    if wheel_angle > 0.5:
        wheel_angle = 0.5
    elif wheel_angle < -0.5:
        wheel_angle = -0.5
    # update steering angle
    angle = wheel_angle

def change_steer_angle(inc):
    global manual_steering
    # Apply increment
    new_manual_steering = manual_steering + inc
    # Validate interval
    if new_manual_steering <= 25.0 and new_manual_steering >= -25.0:
        manual_steering = new_manual_steering
        set_steering_angle(manual_steering * 0.02)
    # Debugging
    if manual_steering == 0:
        print("going straight")
    else:
        turn = "left" if steering_angle < 0 else "right"
        print("turning {} rad {}".format(str(steering_angle),turn))

def preprocess_image(image):
    """Resize and normalize image"""
    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize
    img_resized = cv2.resize(img_rgb, (IMG_WIDTH, IMG_HEIGHT))
    # Normalize
    return img_resized / 255.0

def predict_steering(image, model):
    # Preprocess image for model
    img = preprocess_image(image)
    img = np.expand_dims(img, axis=0)
    
    # Get prediction
    predicted_angle = model.predict(img)[0][0]
    
    # RESTORED: Print model results for debugging
    print(f"Raw predicted angle: {predicted_angle:.4f}") 
    
    # Determine steering direction
    if predicted_angle > 0: 
        direction = "right" 
    elif predicted_angle < 0: 
        direction = "left" 
    else: 
        direction = "straight" 
     
    print(f"Steering direction: {direction}") 
    
    return predicted_angle

def detect_pedestrian(image, pedestrian_model):
    """Detect if there is a pedestrian in the image"""
    # Convert to grayscale
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Extract HOG features
    img_hog_feature, _ = hog(gray_img,
                            orientations=11,
                            pixels_per_cell=(16,16),
                            cells_per_block=(2,2),
                            transform_sqrt=False,
                            visualize=True,
                            feature_vector=True)
    
    # Reshape features for prediction
    img_hog_feature = img_hog_feature.reshape(1, -1)
    
    # Make prediction
    prediction = pedestrian_model.predict(img_hog_feature)
    
    # Print result
    if prediction[0] == 1:
        print("!! WARNING - Person detected !!")
    
    return prediction[0]

# main
def main():
    # Create the Robot instance.
    robot = Car()
    driver = Driver()

    try:
        # Load the steering model with custom loss
        model = tf.keras.models.load_model(MODEL_PATH, custom_objects={'mse': MeanSquaredError()})
        print("Steering model loaded successfully")
        
        # Load pedestrian detection model
        pedestrian_model = joblib.load(PEDESTRIAN_MODEL_PATH)
        print("Pedestrian detection model loaded successfully")
    except FileNotFoundError as e:
        print(f"Error: Model file not found: {str(e)}")
        print("Please ensure the model files exist in the specified paths")
        return
    except Exception as e:
        print(f"Error loading models: {str(e)}")
        return

    # Get the time step of the current world.
    timestep = int(robot.getBasicTimeStep())

    # Create camera instance
    camera = robot.getDevice("camera")
    camera.enable(timestep)

    # Get and enable front sensor (radar is called "radar(1)")
    front_radar = robot.getDevice("radar(1)")
    front_radar.enable(timestep)

    # Create keyboard instance
    keyboard = Keyboard()
    keyboard.enable(timestep)

    # processing display
    display_img = None # Initialize to None
    try:
        display_img = Display("display_image")
    except RuntimeError:
        print("Warning: 'display_image' device not found. Image display will be skipped.")
    
    # Define thresholds for radar
    STOP_DISTANCE = 0.9  # Meters: stop if target is closer than this
    SLOW_DOWN_DISTANCE = 20 # Meters: slow down if target is closer than this
    SLOW_DOWN_SPEED = 10.0 # km/h: speed when slowing down
    REVERSE_DISTANCE = 7.0 # Meters: start reversing if target is closer than this
    REVERSE_SPEED = -5.0 # km/h: speed when reversing

    # Initial cruising speed
    normal_cruising_speed = initial_speed # Renamed for clarity, using the global initial_speed value

    while robot.step() != -1:
        # Get and process image
        image = get_image(camera)
        #grey_image = greyscale_cv2(image)
        #if display_img:
           #display_image(display_img, grey_image)

        # --- Pedestrian Detection (Warning Only) ---
        detect_pedestrian(image, pedestrian_model)

        # --- Speed Control Logic (Radar Priority) ---
        target_speed = normal_cruising_speed # Default speed is the cruising speed

        radar_targets = front_radar.getTargets()
        
        if len(radar_targets) > 0:
            # Find the closest target
            closest_target_distance = float('inf')
            for target in radar_targets:
                if target.distance < closest_target_distance and target.distance > 1.0: # Ignore readings of 1.0m
                    closest_target_distance = target.distance
            
            # Only adjust speed if we have a valid target distance
            if closest_target_distance != float('inf'):
                print(f"Closest radar target distance: {closest_target_distance:.2f}m")

                if closest_target_distance < STOP_DISTANCE:
                    target_speed = 0.0  # Complete stop
                    print("!! EMERGENCY STOP - Obstacle critically close !!")
                elif closest_target_distance < REVERSE_DISTANCE:
                    target_speed = REVERSE_SPEED  # Go backwards
                    print(f"!! REVERSING - Moving backwards at {abs(REVERSE_SPEED):.1f} km/h !!")
                elif closest_target_distance < SLOW_DOWN_DISTANCE:
                    # Calculate speed reduction proportional to distance
                    distance_factor = (closest_target_distance - STOP_DISTANCE) / (SLOW_DOWN_DISTANCE - STOP_DISTANCE)
                    target_speed = min(SLOW_DOWN_SPEED * distance_factor, normal_cruising_speed)
                    print(f"!! CAUTION - Reducing speed to {target_speed:.1f} km/h !!")
            else:
                normal_cruising_speed = min(normal_cruising_speed + speed_increment, MAX_SPEED)
                target_speed = normal_cruising_speed
                print(f"No valid obstacles detected. Speed: {target_speed:.1f} km/h")
        else:
            normal_cruising_speed = min(normal_cruising_speed + speed_increment, MAX_SPEED)
            target_speed = normal_cruising_speed
            print(f"No obstacles detected. Speed: {target_speed:.1f} km/h")

        # Check keyboard input first
        key = keyboard.getKey()
        if key != -1:  # If any key is pressed
            if key == keyboard.UP:
                target_speed += 5.0
                print("up")
            elif key == keyboard.DOWN:
                target_speed -= 5.0
                print("down")
            elif key == keyboard.RIGHT:
                change_steer_angle(+1)
                print("right")
            elif key == keyboard.LEFT:
                change_steer_angle(-1)
                print("left")
            elif key == ord('R'):  # Add reverse control with 'R' key
                target_speed = -10.0  # Set negative speed for reverse
                print("reversing")
        else:
            # Only use model prediction if no key is pressed
            predicted_angle = predict_steering(image, model)
            set_steering_angle(predicted_angle)

        # Apply controls
        driver.setSteeringAngle(angle)
        driver.setCruisingSpeed(target_speed)

if __name__ == "__main__":
    main()